In [3]:
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
from datetime import datetime
from os.path import isfile
import requests
import plotly.express as px
import dash_bootstrap_components as dbc
from IPython.display import Image

In [4]:
at = requests.get("https://api.data.gov.in/resource/3b01bcb8-0b14-4abf-b6f2-c1bfd384ba69?api-key=579b464db66ec23bdd0000017032584c46ea435260efffec57a7a780&format=json&offset=0&limit=1000")

In [5]:
json_dat = at.json()
ab = pd.DataFrame(json_dat["records"])
ab.set_index("id")
ab.head(80)

,id,country,state,city,station,last_update,pollutant_id,pollutant_min,pollutant_max,pollutant_avg,pollutant_unit
0,1,India,Andhra_Pradesh,Amaravati,"Secretariat, Amaravati - APPCB",21-06-2020 08:00:00,PM2.5,12,40,24,NA
1,2,India,Andhra_Pradesh,Amaravati,"Secretariat, Amaravati - APPCB",21-06-2020 08:00:00,PM10,22,59,40,NA
2,3,India,Andhra_Pradesh,Amaravati,"Secretariat, Amaravati - APPCB",21-06-2020 08:00:00,NO2,8,20,12,NA
3,4,India,Andhra_Pradesh,Amaravati,"Secretariat, Amaravati - APPCB",21-06-2020 08:00:00,NH3,2,3,2,NA
4,5,India,Andhra_Pradesh,Amaravati,"Secretariat, Amaravati - APPCB",21-06-2020 08:00:00,SO2,1,24,15,NA
...,...,...,...,...,...,...,...,...,...,...,...
75,76,India,Bihar,Patna,"Samanpura, Patna - BSPCB",21-06-2020 08:00:00,PM10,43,75,59,NA
76,77,India,Bihar,Patna,"Samanpura, Patna - BSPCB",21-06-2020 08:00:00,NO2,6,35,24,NA
77,78,India,Bihar,Patna,"Samanpura, Patna - BSPCB",21-06-2020 08:00:00,NH3,1,1,1,NA
78,79,India,Bihar,Patna,"Samanpura, Patna - BSPCB",21-06-2020 08:00:00,CO,19,32,21,NA


In [6]:
train = ab.copy()
train.pollutant_min = pd.to_numeric(train.pollutant_min,errors = "coerce")
train.pollutant_max = pd.to_numeric(train.pollutant_max,errors = "coerce")
train.pollutant_avg = pd.to_numeric(train.pollutant_avg,errors = "coerce")

In [7]:
train = train.dropna(subset = ["pollutant_min"])

In [8]:
states = list(train["state"].unique())
print(states)

['Andhra_Pradesh', 'Assam', 'Bihar', 'Chandigarh', 'Delhi', 'Gujarat', 'Haryana', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Meghalaya', 'Mizoram', 'Odisha', 'Punjab', 'Rajasthan']


In [9]:
st_ct = {}
for i in states:
    st_ct[i] = (list(train.loc[train["state"]==i,"city"].unique()))
    

In [10]:
print(st_ct)

{'Andhra_Pradesh': ['Amaravati', 'Rajamahendravaram', 'Tirupati', 'Visakhapatnam'], 'Assam': ['Guwahati'], 'Bihar': ['Gaya', 'Hajipur', 'Muzaffarpur', 'Patna'], 'Chandigarh': ['Chandigarh'], 'Delhi': ['Delhi'], 'Gujarat': ['Ahmedabad', 'Gandhinagar', 'Vapi', 'Vatva'], 'Haryana': ['Ambala', 'Bahadurgarh', 'Ballabgarh', 'Bhiwani', 'Dharuhera', 'Faridabad', 'Gurugram', 'Jind', 'Kaithal', 'Karnal', 'Kurukshetra', 'Mandikhera', 'Manesar', 'Narnaul', 'Palwal', 'Panchkula', 'Panipat', 'Rohtak', 'Sirsa', 'Sonipat', 'Yamunanagar'], 'Jharkhand': ['Jorapokhar'], 'Karnataka': ['Bagalkot', 'Bengaluru', 'Chikkaballapur', 'Chikkamagaluru', 'Hubballi', 'Kalaburgi', 'Mysuru', 'Ramanagara', 'Vijayapura', 'Yadgir'], 'Kerala': ['Eloor', 'Kannur', 'Kochi', 'Kollam', 'Kozhikode', 'Thiruvananthapuram'], 'Madhya Pradesh': ['Bhopal', 'Damoh', 'Dewas', 'Gwalior', 'Indore', 'Jabalpur', 'Katni', 'Maihar', 'Mandideep', 'Pithampur', 'Ratlam', 'Sagar', 'Satna', 'Singrauli', 'Ujjain'], 'Maharashtra': ['Aurangabad', '

In [94]:
app = dash.Dash(__name__,external_stylesheets=[dbc.themes.SLATE])

In [95]:
colors = {
    'background': '#111121',
    'text': '#7FDBFF'
}


In [96]:
app.layout = html.Div([

    html.H1("AIR QUALITY INDEX", 
            style={'text-align': 'center',
                   'color':colors['text']}),
     html.Br(),html.Br(),
    
    html.H2("Check Air Quality Index of Your City",style={'color': colors['text'],'padding':'5pt'}),
    dcc.Dropdown(id='slct_state',
                 options=[{"label":x,"value":x} for x in st_ct.keys()],
                 multi=False,
                 value='Delhi',
                 placeholder="Select  State",
                 style={'width': "40%",'color':"black",'padding-left':'10pt'}
                 ),
    html.Br(),
    dcc.Dropdown(id='slct_city',
                 multi=False,
                 value='Delhi',
                 placeholder="Select a City",
                 style={'width': "40%",'color':"black",'padding-left':'10pt'}
                 ),
     html.Br(),html.Br(),

    html.H3(html.Div(id='output_container', children=[]),style={'color': colors['text'],'padding-left':'10pt'}),
    html.Br(),

    
    dcc.Graph(id='my_bee_map',
             style={'width': "60%",'display':'inline-block'}), 
    
    html.Img(src= "https://w.ndtvimg.com/sites/3/2019/12/18122812/air_pollution_standards_cpcb.png",style={'padding':'10pt','width': "40%",'vertical-align':'top','display':'inline-block'})
    
    
    

])

In [97]:
@app.callback(
    Output(component_id='slct_city', component_property='options'),
    [Input(component_id='slct_state', component_property='value')]
)
def set_cities_options(selected_state):
    print(selected_state)
    return [{'label': i, 'value': i} for i in st_ct[selected_state]]

In [98]:
@app.callback(
    Output(component_id='slct_city', component_property='value'),
    [Input(component_id='slct_city', component_property='options')]
)
def set_cities_value(available_options):
    #print(available_options[0]['value'])
    return available_options[0]['value']

In [99]:
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure')],
    [Input(component_id='slct_city', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The AQI Of {} is:".format(option_slctd)
    df =  train[train["city"]==option_slctd]
    fig = px.bar(df, x='pollutant_id', y='pollutant_avg',color ="station", barmode = 'group', height=400,template='plotly_dark')
    return container, fig


In [100]:
app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jun/2020 09:59:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2020 09:59:33] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2020 09:59:33] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2020 09:59:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2020 09:59:33] "POST /_dash-update-component HTTP/1.1" 200 -


Delhi
Delhi
<class 'str'>


127.0.0.1 - - [21/Jun/2020 09:59:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2020 10:00:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2020 10:00:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2020 10:00:13] "POST /_dash-update-component HTTP/1.1" 200 -


Rajasthan
Ajmer
<class 'str'>


...

Meghalaya
Shillong
<class 'str'>


127.0.0.1 - - [21/Jun/2020 10:00:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2020 10:00:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2020 10:00:41] "POST /_dash-update-component HTTP/1.1" 200 -


Kerala
Eloor
<class 'str'>


127.0.0.1 - - [21/Jun/2020 10:00:48] "POST /_dash-update-component HTTP/1.1" 200 -


Kollam
<class 'str'>
